In [150]:
# Distribuição de Poisson

import math

prob_gols = 0
media = 0.59796

(math.exp(-media)*media**prob_gols)/math.factorial(prob_gols)

0.5499323545758446

In [1]:
# manipulando tabela do brasileirão série A a partir de 2012
# para verificação de estratégia de Poisson - ainda falta terminar

import csv 
import statistics as st
import math

linha_inicial = False
time_casa = ''
time_fora = ''
num_jogos = 0
num_gols = 0
media_gols = 0
poisson = 0
with open('BRA.csv', 'r', encoding = 'utf-8', newline = '') as al:
    arq1 = csv.DictReader(al)
    
    for linha in arq1:
        if (linha['Home'] == 'Sport Recife' and linha['Away'] == 'Atletico-MG' and linha_inicial == False):
            linha_inicial = True
        if linha_inicial == False:
            num_jogos += 1
            num_gols += int(linha['HG']) + int(linha['AG'])
            media_gols = num_gols/num_jogos
        if (linha_inicial == True) and (linha['Date'].split('/')[2] == '2012'):
           
            
            num_jogos += 1
            num_gols += int(linha['HG']) + int(linha['AG'])
            media_gols = num_gols/num_jogos
            time_casa = linha['Home']
            time_fora = linha['Away']        

In [33]:
# Distribuição de Poisson para resultados de futebol

import math
import numpy as np

mgf_tcasa = 2 
mgs_tcasa = 0.57
mgf_tfora = 1.29
mgs_tfora = 0.86
mgf_ccasa = 1.14
mgs_ccasa = 0.92
mgf_cfora = 0.92
mgs_cfora = 1.14

media_getc = mgf_tcasa / mgf_ccasa * mgs_tfora / mgs_cfora * mgf_ccasa
media_getf = mgf_tfora / mgf_cfora * mgs_tcasa / mgs_ccasa * mgf_cfora

# p/ prob. do número de gols
probf_casa = [0,0,0,0,0,0,0,0,0,0,0]
probf_fora = [0,0,0,0,0,0,0,0,0,0,0]
probs_casa = [0,0,0,0,0,0,0,0,0,0,0]
probs_fora = [0,0,0,0,0,0,0,0,0,0,0]

# p/ prob. de vitória, empate e derrota
probv_casa = 0
probv_fora = 0
prob_empate = 0

# p/ prob. dupla chance
dupla_casa = 0
dupla_fora = 0

# p/ prob. empate anula
emp_ac = 0
emp_af = 0

# p/ prob. under e over
under_05 = 0
over_05 = 0
under_15 = 0
over_15 = 0
under_25 = 0
over_25 = 0
under_35 = 0
over_35 = 0

# p/ prob. ambas marcam
ambas_sim = 0
ambas_nao = 0

prob_placar = np.zeros((11,11), dtype=np.float64)
placar_ordenado = []
pos_placar_ordenado = []
posicao = 0
cont = 0

# prob. de gols do time da casa e fora
for i in range(11):
    probf_casa[i] = (math.exp(-media_getc)*media_getc**i)/math.factorial(i)
    probf_fora[i] = (math.exp(-media_getf)*media_getf**i)/math.factorial(i)

# prob. de placares do jogo; prob. de 1x2; prob. de under e over
# i -> num de gols do time da casa; j -> num de gols do time de fora
for i in range(11):
    for j in range(11):
        # prob. de 1x2
        if i > j:
            probv_casa += probf_casa[i] * probf_fora[j] * 100
        elif j > i:
            probv_fora += probf_casa[i] * probf_fora[j] * 100
        else: 
            prob_empate += probf_casa[i] * probf_fora[j] * 100
        
        # prob. de under e over
        # under/over 0.5
        if i + j <= 0:
            under_05 += probf_casa[i] * probf_fora[j] * 100
        
        # under/over 1.5
        if i + j <= 1:
            under_15 += probf_casa[i] * probf_fora[j] * 100
            
        # under/over 2.5
        if i + j <= 2:
            under_25 += probf_casa[i] * probf_fora[j] * 100
            
        # under/over 3.5
        if i + j <= 3:
            under_35 += probf_casa[i] * probf_fora[j] * 100
            
        # prob. ambas marcam
        if (i > 0) and (j > 0):
            ambas_sim += probf_casa[i] * probf_fora[j] * 100
         
        # prob. de placar exato
        prob_placar[i][j] = probf_casa[i] * probf_fora[j] * 100
        placar_ordenado.append(prob_placar[i][j])
        pos_placar_ordenado.append(placar_ordenado[cont])
        cont += 1
        # print(f'{i} x {j} -> {prob_placar[i][j]:.2f}%')

# prob. dupla chance
dupla_casa = (probv_casa + prob_empate) 
dupla_fora = (probv_fora + prob_empate)
        
# prob. empate anula
emp_ac = probv_casa/(probv_casa + probv_fora) * 100
emp_af = 100 - emp_ac

# prob. over 0.5
over_05 = 100 - under_05

# prob. over 1.5
over_15 = 100 - under_15

# prob. over 2.5
over_25 = 100 - under_25

# prob. over 3.5
over_35 = 100 - under_35

# prob. ambas marcam não
ambas_nao = 100 - ambas_sim
        
placar_ordenado.sort(reverse = True)

print(f'\nPlacar Ordenado\n')
# for i in range(121): 121 é o máximo. Nesse range pode-se escolher quantos resultados corretos exibir
for i in range(10):
    posicao = pos_placar_ordenado.index(placar_ordenado[i])
    
    # Resultados Corretos
    print(f'{int(posicao/11)} x {posicao%11} -> {placar_ordenado[i]:.2f}%')

# 1x2
print(f'\nprob. de vitória casa => {probv_casa:.2f}%')
print(f'prob. de vitória visitante => {probv_fora:.2f}%')
print(f'prob. de empate: {prob_empate:.2f}%')

# dupla chance
print(f'\nprob. dupla chance casa => {dupla_casa:.2f}%')
print(f'prob. dupla chance visitante => {dupla_fora:.2f}%')

# empate anula
print(f'\nprob. empate anula casa => {emp_ac:.2f}%')
print(f'prob. empate anula visitante: => {emp_af:.2f}%')

# under/over 0.5
print(f'\nprob. under 0.5 => {under_05:.2f}%')
print(f'prob. over 0.5 => {over_05:.2f}%')

# under/over 1.5
print(f'\nprob. under 1.5 => {under_15:.2f}%')
print(f'prob. over 1.5 => {over_15:.2f}%')

# under/over 2.5
print(f'\nprob. under 2.5 => {under_25:.2f}%')
print(f'prob. over 2.5 => {over_25:.2f}%')

# under/over 3.5
print(f'\nprob. under 3.5 => {under_35:.2f}%')
print(f'prob. over 3.5 => {over_35:.2f}%')

# ambas marcam
print(f'\nprob. ambas marcam sim => {ambas_sim:.2f}%')
print(f'prob. ambas marcam não => {ambas_nao:.2f}%')


Placar Ordenado

1 x 0 -> 15.01%
1 x 1 -> 11.99%
2 x 0 -> 11.32%
0 x 0 -> 9.95%
2 x 1 -> 9.05%
0 x 1 -> 7.95%
3 x 0 -> 5.69%
1 x 2 -> 4.79%
3 x 1 -> 4.55%
2 x 2 -> 3.62%

prob. de vitória casa => 54.07%
prob. de vitória visitante => 19.86%
prob. de empate: 26.08%

prob. dupla chance casa => 80.14%
prob. dupla chance visitante => 45.93%

prob. empate anula casa => 73.14%
prob. empate anula visitante: => 26.86%

prob. under 0.5 => 9.95%
prob. over 0.5 => 90.05%

prob. under 1.5 => 32.90%
prob. over 1.5 => 67.10%

prob. under 2.5 => 59.39%
prob. over 2.5 => 40.61%

prob. under 3.5 => 79.77%
prob. over 3.5 => 20.23%

prob. ambas marcam sim => 42.86%
prob. ambas marcam não => 57.14%
